In [ ]:
from ta.momentum import RSIIndicator
from ta.trend import MACD
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

# Downloading the historical data
def get_data(ticker="AAPL", period="2y"):
    df = yf.download(ticker, period=period)
    df = df[['Close']].copy()
    return df

# Computing the indicators
def add_indicators(df):
    df['RSI'] = RSIIndicator(close=df['Close']).rsi()
    macd = MACD(close=df['Close'])
    df['MACD'] = macd.macd_diff()
    return df.dropna()

Preparing the financial data for LSTM model
def prepare_data(df, window=30):
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(df)

    X, y = [], []
    for i in range(window, len(scaled)):
        X.append(scaled[i-window:i])
        y.append(scaled[i, 0])

    return np.array(X), np.array(y), scaler

# Building the LSTM Model
def build_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(units=32))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Backtesting the strategy
def backtest(predicted, actual):
    df_bt = pd.DataFrame({'Predicted': predicted.flatten(), 'Actual': actual.flatten()})
    df_bt['Signal'] = np.where(df_bt['Predicted'].shift(1) < df_bt['Actual'].shift(1), 1, -1)
    df_bt['Returns'] = df_bt['Signal'] * df_bt['Actual'].pct_change()
    df_bt['Cumulative'] = (1 + df_bt['Returns']).cumprod()

    sharpe = df_bt['Returns'].mean() / df_bt['Returns'].std() * np.sqrt(252)
    roi = df_bt['Cumulative'].iloc[-1] - 1
    max_dd = (df_bt['Cumulative'].cummax() - df_bt['Cumulative']).max()

    print(f"Sharpe Ratio: {sharpe:.2f}")
    print(f"Return on Investment (ROI): {roi*100:.2f}%")
    print(f"Maximum Drawdown: {max_dd*100:.2f}%")

    df_bt['Cumulative'].plot(title="Strategy Cumulative Returns")
    plt.show()

# Main Execution
df = get_data("AAPL")
df = add_indicators(df)
X, y, scaler = prepare_data(df[['Close', 'RSI', 'MACD']])

# Train/test split
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

model = build_model(X_train.shape[1:])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Predictions
predicted = model.predict(X_test)
actual = y_test.reshape(-1, 1)
predicted_prices = scaler.inverse_transform(np.hstack((predicted, np.zeros((len(predicted), 2)))))[:,0]
actual_prices = scaler.inverse_transform(np.hstack((actual, np.zeros((len(actual), 2)))))[:,0]

# Backtesting the strategy
backtest(predicted_prices, actual_prices)